# 7.2.1 콜백을 사용하여 모델의 훈련 과정 제어하기

1. 콜백: 모델의 fit() method가 호출될 때 전달되는 객체
2. 콜백을 사용하는 사례
    1. 모델 체크포인트 저장: 모델의 현재 가중치 저장
    2. 조기 종료(early stopping): 검증 손실이 향상되지 않을 때 훈련 중지
    3. 훈련 중 하이퍼파라미터 값을 동적으로 조정: ex. optimizer lr
    4. 훈련과 검증 지표를 로그에 기록 or 모델이 학습한 표현이 업데이트될 때마다 시각화

3. Model Checkpoint와 EarlyStopping 콜백 코드 예시
    ```python
    import keras

    callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=1, ), #2 epcoh 동안 정확도가 향상되지 않으면 훈련 중지
                      keras.callbacks.ModelCheckpoint(filepath='my_model.h5', monitor='val_loss', save_best_only=True),] #val_loss가 좋아지지 않으면 모델 파일을 덮어쓰지 않음

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) #정확도를 모니터링하므로 모델 지표에 포함돼야 함

    model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val)) #콜백이 val acc & loss를 모니터링 하기에 val data 매개변수에 val data를 전달해야 함
    ```

4. ReduceLROnPlateau 콜백
    1. 검증 손실이 향상되지 않을 때 학습률을 작게 할 수 있음
    2. 코드 예시
    ```python
    callbacks_list = [keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=10,)] #콜백 호출시 학습률을 10배로 줄임
    
    model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))
    ```

5. 자신만의 콜백 만들기
    1. 매 epoch의 끝에서 검증 세트의 첫 번째 샘플로 모델에 있는 모든 층의 활성화 출력을 넘파이 배열로 계산하여 디스크에 저장하는 자작 콜백의 예시 코드
    ```python
    import keras
    import numpy as np

    class ActivationLogger(keras.callbacks.Callback):
            def set_model(self, model):
                self.model = model
                layer_outputs = [layer.output for layer in model.layers]
                self.activations_model = keras.models.Model(model.input, layer_outputs)
                
            def on_epoch_end(self, epoch, logs=None):
                if self.validation_data is None:
                    raise RuntimeError('Requires validation_data.')
            
                validation_sample = self.validation_data[0][0:1] #Pick 검증 데이터의 첫 번째 샘플
                activations = self.activations_model.predict(validation_sample)
                f = open('activation_at_epoch_' + str(epoch) + '.npz', 'wb')
                np.savez(f, activations)
                f.close()
    ```

# 7.2.2 텐서보드 소개: 텐서플로의 시각화 프레임워크

1. 텐서보드: 텐서플로와 함께 제공되는 브라우저 기반 시각화 도구
2. 기능
    1. 훈련하는 동안 측정 지표를 시각적으로 모니터링
    2. 모델 구조 시각화
    3. 활성화 출력과 그래디언트 히스토그램 생성
    4. 3D로 임베딩 표현

In [1]:
#텐서보드를 사용한 텍스트 분류 모델
import keras
import tensorflow as tf
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, 500, 128)          256000    
                                                                 
 conv1d (Conv1D)             (None, 494, 32)           28704     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 98, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 92, 32)            7200      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 3

In [3]:
#텐서보드 로그 파일을 위한 디렉토리 생성
#! mkdir my_log_dir

In [4]:
#텐서보드 콜백과 함께 모델 훈련하기
callbacks = [keras.callbacks.TensorBoard(log_dir='my_log_dir', histogram_freq=1, embeddings_freq=1,)] #1 epoch마다 활성화 출력의 히스토그램과 임베딩 데이터를 기록
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks)

Epoch 1/20
157/157 [==============================] - 21s 129ms/step - loss: 0.5866 - acc: 0.6935 - val_loss: 0.4243 - val_acc: 0.8498
Epoch 2/20
157/157 [==============================] - 20s 128ms/step - loss: 0.4277 - acc: 0.8572 - val_loss: 0.4740 - val_acc: 0.8496
Epoch 3/20
157/157 [==============================] - 20s 125ms/step - loss: 0.3645 - acc: 0.8831 - val_loss: 0.4822 - val_acc: 0.8472
Epoch 4/20
157/157 [==============================] - 20s 127ms/step - loss: 0.3302 - acc: 0.9011 - val_loss: 0.5647 - val_acc: 0.8476
Epoch 5/20
157/157 [==============================] - 20s 127ms/step - loss: 0.2765 - acc: 0.9205 - val_loss: 0.5738 - val_acc: 0.8688
Epoch 6/20
157/157 [==============================] - 21s 133ms/step - loss: 0.2490 - acc: 0.9351 - val_loss: 0.6295 - val_acc: 0.8666
Epoch 7/20
157/157 [==============================] - 21s 133ms/step - loss: 0.2030 - acc: 0.9535 - val_loss: 0.6740 - val_acc: 0.8636
Epoch 8/20
157/157 [==============================] - 2

In [8]:
from keras.utils import plot_model

plot_model(model, to_file='model.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
plot_model(model, show_shapes=True, to_file='model2.png')